In [25]:
import warnings
import os
import gspread

import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from pymisp import PyMISP, MISPAttribute
from dotenv import load_dotenv

In [30]:
load_dotenv()

misp_api_key = os.getenv("MISP_API_Key")

1. Creation of a PowerAutomate Workflow From *'Save a Tweets to a Google Sheet'* Template
2. Configuring a Google Cloud service to interact with the Google Sheet from the Python code.

In [3]:
gc = gspread.service_account(filename=r".\phishingioc-keys.json")
sh = gc.open("PhishingIoCs")
worksheet = sh.sheet1

In [4]:
# Regex patterns
patternIP4 = r"(?P<ip4>[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3})"
patternUrl = r"(?P<url>hxxps?://[^\s]+)"
patternEma = r"(?P<email>[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7})"
patternDom = r"(?P<domain>(?:[a-z0-9](?:[a-z0-9-]{0,61}[a-z0-9])?\[\.\])+[a-z0-9][a-z0-9-]{0,61}[a-z0-9])"
patternTLD = r"(?P<tld>\[\.\]([^.]*)$)"

In [5]:
# Tweet Text/Body
IoCRaw_list = worksheet.col_values(3)
IoC_raw = pd.DataFrame(IoCRaw_list[1:], columns = ['TweetBody'])

In [6]:
# Tweet Id
Tid_list = worksheet.col_values(7)
IoC_df = pd.DataFrame(Tid_list[1:], columns = ['TweetID'])

In [7]:
# Extract IoCs
IoC_df['IoCURLs'] = IoC_raw.TweetBody.str.extract(patternUrl)
IoC_df['IoCIPv4s'] = IoC_raw.TweetBody.str.extract(patternIP4)
IoC_df['IoCEmails'] = IoC_raw.TweetBody.str.extract(patternEma)
IoC_df['IoCDomains'] = IoC_raw.TweetBody.str.extract(patternDom)
IoC_df['IoCTLDs'] = IoC_df.IoCDomains.str.extract(patternTLD).tld.replace(r"(\[\.\]([0-9]*)$)", np.nan, regex=True)

In [8]:
IoC_df

,TweetID,IoCURLs,IoCIPv4s,IoCEmails,IoCDomains,IoCTLDs
0,1629050806971297792,hxxp://rogil75702[.]temp[.]swtest[.]ru/c49f9,NaN,NaN,rogil75702[.]temp[.]swtest[.]ru,[.]ru
1,1629058798240337924,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site,[.]site
2,1629058762970365954,hxxps://paypay-login[.]huahuayufeng[.]com/all/...,155.94.128.125,NaN,paypay-login[.]huahuayufeng[.]com,[.]com
3,1629058784483020800,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site,[.]site
4,1629072062672408576,hxxps://metamasek.cc/,103.149.92.146,NaN,NaN,NaN
...,...,...,...,...,...,...
288,1632711875795496961,hxxps://blur.web3claim[.]org/,NaN,NaN,web3claim[.]org,[.]org
289,1632727015349444612,NaN,NaN,NaN,ssvnetwork[.]web3claim[.]org,[.]org
290,1632726475349557250,NaN,NaN,NaN,NaN,NaN
291,1632742046011215874,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop NaN Values on IoCs Columns
IoC_df_NNAN = IoC_df.dropna(subset=['IoCURLs', 'IoCIPv4s', 'IoCEmails', 'IoCDomains', 'IoCTLDs'], how='all').reset_index(drop=True)
IoC_df_NNAN

,TweetID,IoCURLs,IoCIPv4s,IoCEmails,IoCDomains,IoCTLDs
0,1629050806971297792,hxxp://rogil75702[.]temp[.]swtest[.]ru/c49f9,NaN,NaN,rogil75702[.]temp[.]swtest[.]ru,[.]ru
1,1629058798240337924,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site,[.]site
2,1629058762970365954,hxxps://paypay-login[.]huahuayufeng[.]com/all/...,155.94.128.125,NaN,paypay-login[.]huahuayufeng[.]com,[.]com
3,1629058784483020800,hxxps://stemcommunitty[.]site/,NaN,NaN,stemcommunitty[.]site,[.]site
4,1629072062672408576,hxxps://metamasek.cc/,103.149.92.146,NaN,NaN,NaN
...,...,...,...,...,...,...
108,1632483462174109701,hxxp://www[.]mobildenizv3deniz[.]top[.]mobilde...,NaN,NaN,www[.]mobildenizv3deniz[.]top[.]mobildenizv2de...,[.]top
109,1632514094249984000,NaN,NaN,NaN,www[.]terminal7[.]sbs,[.]sbs
110,1632650596657315840,hxxps://fe3f1f5c-485f-4272-8520-63fea74045fc[....,NaN,NaN,fe3f1f5c-485f-4272-8520-63fea74045fc[.]id[.]re...,[.]co
111,1632711875795496961,hxxps://blur.web3claim[.]org/,NaN,NaN,web3claim[.]org,[.]org


In [10]:
# Types Of IoCs by Bar Chart
x = ["Types Of Indicators"]
Sercounts = IoC_df_NNAN[['IoCURLs', 'IoCIPv4s', 'IoCEmails', 'IoCDomains', 'IoCTLDs']].apply(lambda x: sum(x.notnull()))

fig1 = go.Figure(go.Bar(x=x, y=[Sercounts.IoCURLs], name='URLs'))
fig1.add_trace(go.Bar(x=x, y=[Sercounts.IoCIPv4s], name='IPv4s'))
fig1.add_trace(go.Bar(x=x, y=[Sercounts.IoCEmails], name='Emails'))
fig1.add_trace(go.Bar(x=x, y=[Sercounts.IoCDomains], name='Domains'))

fig1.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig1.show()

In [11]:
# TLDs Present in PhishingIoCs Sheet
countTLDs = IoC_df_NNAN.pivot_table(index = ['IoCTLDs'], aggfunc ='size')
countTLDs = pd.DataFrame({'IoCTLDs':countTLDs.index, 'Count':countTLDs.values})

countTLDs.Count.sum()

fig2 = px.pie(values=countTLDs.Count, names=countTLDs.IoCTLDs)
fig2.show()

In [ ]:
# Data/IoCs Enrichment (Virustotal)

## Example Email


3. Create an *Authentification Key* bound to the Publisher user. 

In [12]:
# Add Attribute From a given list to an Event
def add_attribute_fromList(MISPInstance, eventID, List, type):
    for attr in List:
        misp_attribute = MISPAttribute()

        misp_attribute.value = str(attr)
        misp_attribute.category = str("Network activity")
        misp_attribute.type = str(type)
        misp_attribute.add_tag("Phishing")
        misp_attribute.add_tag(type)

        MISPInstance.add_attribute(eventID, misp_attribute)

In [32]:
warnings.filterwarnings("ignore")

# Add Collected IoCs to Local MISP Instance
## Ignoring the certificate verification

misp_url = "https://localhost/"
misp_key = misp_api_key
misp_verifycert = False

myMISPInstance = PyMISP( misp_url, misp_key, misp_verifycert, "json" )

listOfUrls = list(IoC_df_NNAN['IoCURLs'].dropna())
listOfIpv4 = list(IoC_df_NNAN['IoCIPv4s'].dropna())
listOfEmails = list(IoC_df_NNAN['IoCEmails'].dropna())
listOfDomains = list(IoC_df_NNAN['IoCDomains'].dropna())

## Example
add_attribute_fromList(myMISPInstance, "4", listOfUrls, "url")